In [2]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device}...')

Using cuda...


In [3]:
from datasets import load_dataset

fever_plus = load_dataset("tommasobonomo/sem_augmented_fever_nli")

c:\Users\damic\anaconda3\envs\nlp_env2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import pandas as pd
pd.DataFrame(fever_plus["train"][0])

,id,premise,hypothesis,label,wsd,srl
premise,150448,Roman Atwood . He is best known for his vlogs ...,Roman Atwood is a content creator.,ENTAILMENT,"[{'index': 0, 'text': 'Roman', 'pos': 'ADJ', '...","{'tokens': [{'index': 0, 'rawText': 'Roman'}, ..."
hypothesis,150448,Roman Atwood . He is best known for his vlogs ...,Roman Atwood is a content creator.,ENTAILMENT,"[{'index': 0, 'text': 'Roman', 'pos': 'PROPN',...","{'tokens': [{'index': 0, 'rawText': 'Roman'}, ..."


In [14]:
 
print(type(fever_plus))  # Expected: dict
print(type(fever_plus["train"]))  # Expected: list
print(type(fever_plus["train"][0]))  # Expected: dict
print(type(fever_plus["train"][0]["wsd"]))  # Expected: dict or list
print(type(fever_plus["train"][0]["wsd"]["premise"]))  # Expected: dict or list
print(type(fever_plus["train"][0]["wsd"]["premise"]["pos"]))  # Expected: list

<class 'datasets.dataset_dict.DatasetDict'>
<class 'datasets.arrow_dataset.Dataset'>
<class 'dict'>
<class 'dict'>
<class 'list'>


TypeError: list indices must be integers or slices, not str

In [19]:
[print(wsd['pos']) for wsd in fever_plus["train"][0]["wsd"]["premise"]]
#pos_tags = [wsd['pos'] for wsd in fever_plus["train"][0]["wsd"]["premise"]]



ADJ
PROPN
PUNCT
PRON
AUX
ADV
VERB
ADP
PRON
NOUN
PUNCT
SCONJ
PRON
VERB
NOUN
ADP
PRON
NOUN
ADP
DET
ADJ
NOUN
PUNCT
PRON
VERB
NOUN
PUNCT
PUNCT
PUNCT
X
PUNCT
PUNCT
AUX
DET
NOUN
ADP
NUM
NUM
NOUN
CCONJ
NUM
NUM
NOUN
PUNCT
PRON
ADV
AUX
DET
PROPN
NOUN
VERB
PUNCT
PUNCT
PROPN
PUNCT
PUNCT
SCONJ
PRON
VERB
NOUN
PUNCT


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [20]:
# free up CUDA space
torch.cuda.empty_cache()

In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch

# Load the DeBERTa v3 model and tokenizer from Hugging Face
model_name = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Create an NLI pipeline
nli_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer, device=device)


c:\Users\damic\anaconda3\envs\nlp_env2\lib\site-packages\transformers\convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
c:\Users\damic\anaconda3\envs\nlp_env2\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForSequenceClassification were not initialized from

In [22]:
# combine the elements from a fever entry to prepare a feasible input for 
def preprocess_input(entry):

    # Sample premise and hypothesis with POS tagging
    premise = entry["premise"]
    hypothesis = entry["hypothesis"]

    # Corresponding POS tags (this is just a placeholder for the actual POS tags)
    premise_pos = " ".join([wsd['pos'] for wsd in entry["wsd"]["premise"]])             # create a sentence with the list of all the POS tags sepaated by space
    hypothesis_pos = " ".join([wsd['pos'] for wsd in entry["wsd"]["hypothesis"]])       # 

    # Combine text with POS tags
    premise_with_pos = f"{premise} POS: {premise_pos}"
    hypothesis_with_pos = f"{hypothesis} POS: {hypothesis_pos}"

    # Prepare the combined input for the model
    combined_input = f"{premise_with_pos} [SEP] {hypothesis_with_pos}"

    return combined_input


In [23]:
preprocess_input(fever_plus["train"][0])

"Roman Atwood . He is best known for his vlogs , where he posts updates about his life on a daily basis . His vlogging channel , `` RomanAtwoodVlogs '' , has a total of 3.3 billion views and 11.9 million subscribers . He also has another YouTube channel called `` RomanAtwood '' , where he posts pranks . POS: ADJ PROPN PUNCT PRON AUX ADV VERB ADP PRON NOUN PUNCT SCONJ PRON VERB NOUN ADP PRON NOUN ADP DET ADJ NOUN PUNCT PRON VERB NOUN PUNCT PUNCT PUNCT X PUNCT PUNCT AUX DET NOUN ADP NUM NUM NOUN CCONJ NUM NUM NOUN PUNCT PRON ADV AUX DET PROPN NOUN VERB PUNCT PUNCT PROPN PUNCT PUNCT SCONJ PRON VERB NOUN PUNCT [SEP] Roman Atwood is a content creator. POS: PROPN PROPN AUX DET ADJ NOUN PUNCT"

In [25]:

# Use the NLI pipeline to classify the combined input
from tqdm import tqdm

# Assuming `dataset` is a list of input pairs (or a similar structure)
# and `nli_pipeline` is your pipeline for Natural Language Inference (NLI)
results = []

for entry in tqdm(fever_plus["train"]):
    result = nli_pipeline((preprocess_input(entry), entry["label"]))
    results.append(result)





  0%|          | 0/51086 [00:00<?, ?it/s]


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
# The model will return a prediction with a label and a score
label = result[0]['label']
score = result[0]['score']

# Print the result
print(f"Prediction: {label} (Score: {score:.4f})")